# set inputs to train based for different CNN model

In [1]:
feature_file_name = "densenet_features.pkl"
checkpoint_output_file = 'dense-model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
history_file_name = "dense-glove-model_run_history.pkl"

# you dont need to touch any of the below

just run and wait

In [2]:

from utils.helpers import Config
from utils.dataprep import load_set, load_photo_features
from utils.dataprep import load_clean_descriptions, get_tokenizer, max_length_desc
from utils.inputprep import create_sequences, data_generator

c = Config()


feature_file_name = c.ExtractedFeaturesFilePath(feature_file_name)
checkpoint_output_file = checkpoint_output_file
history_file_name = c.ExtractedFeaturesFilePath(history_file_name)

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# 1. load train and dev images features 
train = load_set(c.FlickrTextFilePath("Flickr_8k.trainImages.txt"))
dev = load_set(c.FlickrTextFilePath("Flickr_8k.devImages.txt"))

# use VGG trained features 
train_features = load_photo_features(feature_file_name, train)
dev_features = load_photo_features(feature_file_name, dev)

print("Train ids: %i, and dev ids: %i" % (len(train), len(dev)))
print("Train photos: %i, and dev photos: %i" % (len(train_features), len(dev_features)))

Train ids: 6000, and dev ids: 1000
Train photos: 6000, and dev photos: 1000


In [4]:
train_features["3009047603_28612247d2"].shape[1]

1024

In [5]:
# 2. load clean descriptions for data sets. and load vocabulary 

train_descriptions = load_clean_descriptions(c.ExtractedFeaturesFilePath('descriptions.txt'), train)
dev_descriptions = load_clean_descriptions(c.ExtractedFeaturesFilePath('descriptions.txt'), dev)

print("Train descriptions: %i, and dev descriptions: %i" % (len(train_descriptions), len(dev_descriptions)))


Train descriptions: 6000, and dev descriptions: 1000


In [6]:
# 3. tokensize train and dev sets 

# prepare tokensizer
tokenizer = get_tokenizer(c.TokenizerFilePath) 
max_length = max_length_desc(train_descriptions)

vocab_size = len(tokenizer.word_index) + 1

print( "Tokensizer vocalulary size: %i, Description max length: %i " % (vocab_size, max_length))
# TODO: here we should save the tokenizer for later use, it will be needed when traslating yhat vector to a description 


Tokensizer vocalulary size: 7579, Description max length: 34 


 # 4. define and fit model

In [7]:
from keras.utils import to_categorical, plot_model
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint


###GloVe
from numpy import asarray
from numpy import zeros

In [8]:
# Load GloVe vectors
import numpy
embedding_matrix = numpy.load("GloVe/embedding_matrix.npy")



In [9]:
# define the model
# define the captioning model
def define_model(vocab_size, max_length, input_dim = 4096):
	# feature extractor model
	inputs1 = Input(shape=(input_dim,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 200, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(256)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    
	# now set the embedding weights from GloVe
	model.layers[2].set_weights([embedding_matrix])
	model.layers[2].trainable = False
        
	# compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	model.summary()
	# plot_model(model, to_file='model.png', show_shapes=True)
	return model


In [10]:
input_dim = train_features["3009047603_28612247d2"].shape[1]
model = define_model(vocab_size, max_length, input_dim)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 200)      1515800     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)  

In [11]:
train_data_generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
dev_data_generator = data_generator(dev_descriptions, dev_features, tokenizer, max_length)

In [12]:
# define checkpoint callback
filepath = checkpoint_output_file # 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [13]:
 history = model.fit_generator(
 	train_data_generator,
 	epochs=20,
 	steps_per_epoch=len(train_descriptions),
 	verbose=2, # 1: progress, 2: one line per epoch
 	validation_data= dev_data_generator,
 	validation_steps=len(dev_descriptions),
 	callbacks=[checkpoint])

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 362s - loss: 4.6319 - val_loss: 4.0590

Epoch 00001: val_loss improved from inf to 4.05901, saving model to dense-model-ep001-loss4.650-val_loss4.059.h5
Epoch 2/20
 - 352s - loss: 3.9166 - val_loss: 3.8883

Epoch 00002: val_loss improved from 4.05901 to 3.88828, saving model to dense-model-ep002-loss3.937-val_loss3.888.h5
Epoch 3/20
 - 350s - loss: 3.6857 - val_loss: 3.8209

Epoch 00003: val_loss improved from 3.88828 to 3.82086, saving model to dense-model-ep003-loss3.708-val_loss3.821.h5
Epoch 4/20
 - 351s - loss: 3.5579 - val_loss: 3.8080

Epoch 00004: val_loss improved from 3.82086 to 3.80798, saving model to dense-model-ep004-loss3.581-val_loss3.808.h5
Epoch 5/20
 - 350s - loss: 3.4735 - val_loss: 3.8168

Epoch 00005: val_loss did not improve from 3.80798
Epoch 6/20
 - 351s - loss: 3.4153 - val_loss: 3.8177

Epoch 00006: val_loss did not improve from 3.80798
Epoch 7/20
 - 490s - loss: 3.3765 - val_loss: 3.8316

Epoch 00

KeyboardInterrupt: 

In [14]:
#save history
import pickle

with open(history_file_name, "wb") as pcklfile:
    pickle.dump(history, pcklfile)



NameError: name 'history' is not defined

In [ ]:
import os 



In [ ]:
os.getcwd()